In [1]:
pip install openai

In [2]:
#Module import
from openai import OpenAI

# Hyperparameter
acy_key=""
model="gpt-4o-mini"
role="user"

# Client 생성
client = OpenAI(
  api_key=acy_key
)

# 테스트 이후 함수 정의
def get_completion(prompt, model=model, role=role):
  response = client.chat.completions.create(
    model=model,
    store=True,
    messages=[
      {"role": role, "content": prompt}
    ]
  )
  return response.choices[0].message.content

In [3]:
from google.colab import drive
drive.mount("/content/drive")#, force_remount=True)
%cd "/content/drive/My Drive/2. A.C.Y.(안성환, 명유경, 최지웅)/920. Daily Snippet(Scrum)/"
!pwd
!ls

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1QqbbmvT7C6fsHG5AXsoM357NDoK4DmPE/2. A.C.Y.(안성환, 명유경, 최지웅)/920. Daily Snippet(Scrum)
/content/drive/.shortcut-targets-by-id/1QqbbmvT7C6fsHG5AXsoM357NDoK4DmPE/2. A.C.Y.(안성환, 명유경, 최지웅)/920. Daily Snippet(Scrum)
'2025 GCS 6기 A.C.Y.팀 Daily Snipet.gdoc'
'2025 GCS 6기 A.C.Y. 팀 Monthly Snipet.gdoc'
'2025 GCS 6기 A.C.Y. 팀 Weekly Snipet.gdoc'


In [4]:
from datetime import datetime
import openai
from googleapiclient.discovery import build
from google.oauth2 import service_account

# --- OpenAI 설정 ---
#openai.api_key = "sk-..."  # ← 본인의 API 키로 설정하세요
#model = "gpt-4o"
#role = "user"

def get_completion(prompt, extra_instruction, model=model, role=role):
    full_prompt = f"{prompt}\n\n[요청사항]\n{extra_instruction}"
    response = openai.ChatCompletion.create(
        model=model,
        messages=[{"role": role, "content": full_prompt}]
    )
    return response.choices[0].message.content

# --- Google Docs 설정 ---
SCOPES = ['https://www.googleapis.com/auth/documents']
SERVICE_ACCOUNT_FILE = '/content/drive/My Drive/acy.json'

creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)
docs_service = build('docs', 'v1', credentials=creds)

DOCUMENT_ID = '1o'

# 날짜 변환 함수 ("2025-04-14" → "25.04.14")
def convert_date_for_snippet(iso_date: str) -> str:
    d = datetime.strptime(iso_date, "%Y-%m-%d")
    return d.strftime("25.%m.%d")

# 메인 함수
def generate_daily_feedback(date=datetime.today().strftime("%Y-%m-%d"),
                            instruction="오늘 내용에 대한 칭찬과 개선 피드백을 부탁드립니다."):

    converted_date = convert_date_for_snippet(date)
    doc = docs_service.documents().get(documentId=DOCUMENT_ID).execute()
    content = doc.get('body').get('content')

    team_snippets = {"성환": "", "유경": "", "지웅": ""}
    current_person = None
    capture = False

    for item in content:
        if 'paragraph' in item:
            elements = item['paragraph']['elements']
            text = ''.join([e.get('textRun', {}).get('content', '') for e in elements]).strip()

            # 문서 탭 이름이 나오면 섹션 전환
            if text in team_snippets:
                current_person = text
                capture = False  # 초기화
                continue

            # 날짜 감지 → 해당 섹션 시작
            if current_person and converted_date in text:
                capture = True
                team_snippets[current_person] += text + "\n"
                continue

            if capture and current_person:
                team_snippets[current_person] += text + "\n"

    # GPT 피드백 요청
    feedback_results = {}
    for member, snippet in team_snippets.items():
        if snippet:
            print(f"🔍 {member} 내용 분석 중...")
            feedback = get_completion(snippet, instruction)
            feedback_results[member] = feedback

    # 결과 출력
    for member, feedback in feedback_results.items():
        print(f"\n💬 {member}에 대한 GPT 피드백:\n{feedback}\n")

    return feedback_results



In [5]:
# 실행 예시
generate_daily_feedback("2025-04-14", "피드백은 구체적으로, 단점은 부드럽게 표현해주세요.")


{}